# AICSImage

In [1]:
# Use AICSImage top level entry point

from aicsimageio import AICSImage
AICSImage?

Init signature:
AICSImage(
    image: Union[str, pathlib.Path, numpy.ndarray, dask.array.core.Array, xarray.core.dataarray.DataArray, List[Union[numpy.ndarray, dask.array.core.Array, xarray.core.dataarray.DataArray]], List[Union[str, pathlib.Path]]],
    reader: Optional[Type[aicsimageio.readers.reader.Reader]] = None,
    reconstruct_mosaic: bool = True,
    fs_kwargs: Dict[str, Any] = {},
    **kwargs: Any,
)
Docstring:     
AICSImage takes microscopy image data types (files or arrays) of varying
dimensions ("ZYX", "TCZYX", "CYX") and reads them as consistent 5D "TCZYX"
("Time-Channel-Z-Y-X") ordered array(s). The data and metadata are lazy
loaded and can be accessed as needed.

Parameters
----------
image: types.ImageLike
    A string, Path, fsspec supported URI, or arraylike to read.
reader: Optional[Type[Reader]]
    The Reader class to specifically use for reading the provided image.
    Default: None (find matching reader)
reconstruct_mosaic: bool
    Boolean for setting that da

## Reading

https://allencellmodeling.github.io/aicsimageio/_modules/aicsimageio/readers/ome_zarr_reader.html#OmeZarrReader

In [2]:
# this is the reader class used by AICSImageIO

from aicsimageio import readers
readers.ome_zarr_reader.OmeZarrReader?

Init signature:
readers.ome_zarr_reader.OmeZarrReader(
    image: Union[str, pathlib.Path],
    fs_kwargs: Dict[str, Any] = {},
)
Docstring:     
Wraps the ome-zarr-py API to provide the same aicsimageio Reader API but for
OmeZarr images.

Parameters
----------
image: types.PathLike
    Path to image file to construct Reader for.
fs_kwargs: Dict[str, Any]
    Any specific keyword arguments to pass down to the fsspec created filesystem.
    Default: {}
File:           ~/mambaforge/envs/ome-ngff-readers-writers-aicsimage/lib/python3.10/site-packages/aicsimageio/readers/ome_zarr_reader.py
Type:           ABCMeta
Subclasses:     

In [3]:
# Read from IDR S3

filepath = "s3://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.4/idr0101A/13457539.zarr"

aim = AICSImage(filepath) # fails with no credentials error

NoCredentialsError: Unable to locate credentials

In [4]:
# Read from local fs

filepath = "../data/idr0101"

aim = AICSImage(filepath)

In [5]:
# get image data as numpy array

im_np = aim.get_image_data()
print(im_np.shape)


(18, 6, 11, 189, 190)


In [6]:
# get image data as dask array

im_da = aim.get_dask_stack()
print(im_da.shape)


(1, 18, 6, 11, 189, 190)


In [7]:
# get image data as xarray.DataArray

im_xr = aim.get_xarray_stack()
print(im_xr.shape)


(1, 18, 6, 11, 189, 190)


In [8]:
# metadata encoding

AICSImage?

Init signature:
AICSImage(
    image: Union[str, pathlib.Path, numpy.ndarray, dask.array.core.Array, xarray.core.dataarray.DataArray, List[Union[numpy.ndarray, dask.array.core.Array, xarray.core.dataarray.DataArray]], List[Union[str, pathlib.Path]]],
    reader: Optional[Type[aicsimageio.readers.reader.Reader]] = None,
    reconstruct_mosaic: bool = True,
    fs_kwargs: Dict[str, Any] = {},
    **kwargs: Any,
)
Docstring:     
AICSImage takes microscopy image data types (files or arrays) of varying
dimensions ("ZYX", "TCZYX", "CYX") and reads them as consistent 5D "TCZYX"
("Time-Channel-Z-Y-X") ordered array(s). The data and metadata are lazy
loaded and can be accessed as needed.

Parameters
----------
image: types.ImageLike
    A string, Path, fsspec supported URI, or arraylike to read.
reader: Optional[Type[Reader]]
    The Reader class to specifically use for reading the provided image.
    Default: None (find matching reader)
reconstruct_mosaic: bool
    Boolean for setting that da

# Writing

In [9]:
import dask
import xarray

for im_data in [im_np, im_da, im_xr]:

    if isinstance(im_data, dask.array.Array) or\
        isinstance(im_data, xarray.DataArray):
        im_data_to_write = im_data.squeeze()
    else:
        im_data_to_write = im_data

    aim = AICSImage(im_data_to_write)
    print(aim.shape)
    aim.save('written.zarr')


(18, 6, 11, 189, 190)
(18, 6, 11, 189, 190)
(18, 6, 11, 189, 190)
